# Complex Queries

In this notebook, we will explore more complex queries that can be performed with ``redisvl``

Before running this notebook, be sure to
1. Have installed ``redisvl`` and have that environment active for this notebook.
2. Have a running Redis instance with RediSearch > 2.4 running.

In [1]:
import pickle
from jupyterutils import table_print, result_print

# load in the example data and printing utils
data = pickle.load(open("hybrid_example_data.pkl", "rb"))
table_print(data)

user,age,job,credit_score,office_location,user_embedding
john,18,engineer,high,"-122.4194,37.7749",b'\xcd\xcc\xcc=\xcd\xcc\xcc=\x00\x00\x00?'
derrick,14,doctor,low,"-122.4194,37.7749",b'\xcd\xcc\xcc=\xcd\xcc\xcc=\x00\x00\x00?'
nancy,94,doctor,high,"-122.4194,37.7749",b'333?\xcd\xcc\xcc=\x00\x00\x00?'
tyler,100,engineer,high,"-122.0839,37.3861",b'\xcd\xcc\xcc=\xcd\xcc\xcc>\x00\x00\x00?'
tim,12,dermatologist,high,"-122.0839,37.3861",b'\xcd\xcc\xcc>\xcd\xcc\xcc>\x00\x00\x00?'
taimur,15,CEO,low,"-122.0839,37.3861",b'\x9a\x99\x19?\xcd\xcc\xcc=\x00\x00\x00?'
joe,35,dentist,medium,"-122.0839,37.3861",b'fff?fff?\xcd\xcc\xcc='


In [2]:
schema = {
    "index": {
        "name": "user_index",
        "prefix": "v1",
    },
    "fields": {
        "tag": [{"name": "credit_score"}],
        "text": [{"name": "job"}],
        "numeric": [{"name": "age"}],
        "geo": [{"name": "office_location"}],
        "vector": [{
                "name": "user_embedding",
                "dims": 3,
                "distance_metric": "cosine",
                "algorithm": "flat",
                "datatype": "float32"}
        ]
    },
}


In [3]:
from redisvl.index import SearchIndex

# construct a search index from the schema
index = SearchIndex.from_dict(schema)

# connect to local redis instance
index.connect("redis://localhost:6379")

# create the index (no data yet)
index.create(overwrite=True)

In [4]:
# use the CLI to see the created index
!rvl index listall

19:55:11 [RedisVL] INFO   Indices:
19:55:11 [RedisVL] INFO   1. user_index


In [5]:
# load expects an iterable of dictionaries
index.load(data)

## Executing Hybrid Queries

Hybrid queries are queries that combine multiple types of filters. For example, you may want to search for a user that is a certain age, has a certain job, and is within a certain distance of a location. This is a hybrid query that combines numeric, tag, and geographic filters.

### Tag Filters

Tag filters are filters that are applied to tag fields. These are fields that are not tokenized and are used to store a single categorical value.

In [6]:
from redisvl.query import VectorQuery
from redisvl.query.filter import Tag

t = Tag("credit_score") == "high"

v = VectorQuery([0.1, 0.1, 0.5],
                "user_embedding",
                return_fields=["user", "credit_score", "age", "job", "office_location"],
                filter_expression=t)


results = index.query(v)
result_print(results)

vector_distance,user,credit_score,age,job,office_location
0,john,high,18,engineer,"-122.4194,37.7749"
0.109129190445,tyler,high,100,engineer,"-122.0839,37.3861"
0.158809006214,tim,high,12,dermatologist,"-122.0839,37.3861"
0.266666650772,nancy,high,94,doctor,"-122.4194,37.7749"


In [7]:
# negation
t = Tag("credit_score") != "high"

v.set_filter(t)
result_print(index.query(v))

vector_distance,user,credit_score,age,job,office_location
0,derrick,low,14,doctor,"-122.4194,37.7749"
0.217882037163,taimur,low,15,CEO,"-122.0839,37.3861"
0.653301358223,joe,medium,35,dentist,"-122.0839,37.3861"


### Numeric Filters

Numeric filters are filters that are applied to numeric fields and can be used to isolate a range of values for a given field.

In [8]:
from redisvl.query.filter import Num

numeric_filter = Num("age") > 15

v.set_filter(numeric_filter)

result_print(index.query(v))

vector_distance,user,credit_score,age,job,office_location
0,john,high,18,engineer,"-122.4194,37.7749"
0.109129190445,tyler,high,100,engineer,"-122.0839,37.3861"
0.266666650772,nancy,high,94,doctor,"-122.4194,37.7749"
0.653301358223,joe,medium,35,dentist,"-122.0839,37.3861"


In [9]:
# exact match query
numeric_filter = Num("age") == 14

v.set_filter(numeric_filter)

result_print(index.query(v))

vector_distance,user,credit_score,age,job,office_location
0,derrick,low,14,doctor,"-122.4194,37.7749"


In [10]:
# negation
numeric_filter = Num("age") != 14

v.set_filter(numeric_filter)

result_print(index.query(v))

vector_distance,user,credit_score,age,job,office_location
0,john,high,18,engineer,"-122.4194,37.7749"
0.109129190445,tyler,high,100,engineer,"-122.0839,37.3861"
0.158809006214,tim,high,12,dermatologist,"-122.0839,37.3861"
0.217882037163,taimur,low,15,CEO,"-122.0839,37.3861"
0.266666650772,nancy,high,94,doctor,"-122.4194,37.7749"
0.653301358223,joe,medium,35,dentist,"-122.0839,37.3861"


### Text Filters

Text filters are filters that are applied to text fields. These filters are applied to the entire text field. For example, if you have a text field that contains the text "The quick brown fox jumps over the lazy dog", a text filter of "quick" will match this text field.

In [11]:
from redisvl.query.filter import Text

text_filter = Text("job") == "doctor"

v.set_filter(text_filter)
results = index.query(v)
result_print(results)

vector_distance,user,credit_score,age,job,office_location
0,derrick,low,14,doctor,"-122.4194,37.7749"
0.266666650772,nancy,high,94,doctor,"-122.4194,37.7749"


In [12]:
negate_text_filter = Text("job") != "doctor"

v.set_filter(negate_text_filter)
result_print(index.query(v))

vector_distance,user,credit_score,age,job,office_location
0,john,high,18,engineer,"-122.4194,37.7749"
0.109129190445,tyler,high,100,engineer,"-122.0839,37.3861"
0.158809006214,tim,high,12,dermatologist,"-122.0839,37.3861"
0.217882037163,taimur,low,15,CEO,"-122.0839,37.3861"
0.653301358223,joe,medium,35,dentist,"-122.0839,37.3861"


In [13]:
# fuzzy match filter
fuzzy_text_filter = Text("job") % "doct*"

v.set_filter(fuzzy_text_filter)
result_print(index.query(v))

vector_distance,user,credit_score,age,job,office_location
0,derrick,low,14,doctor,"-122.4194,37.7749"
0.266666650772,nancy,high,94,doctor,"-122.4194,37.7749"


### Geographic Filters

Geographic filters are filters that are applied to geographic fields. These filters are used to find results that are within a certain distance of a given point. The distance is specified in kilometers, miles, meters, or feet. A radius can also be specified to find results within a certain radius of a given point.

In [14]:
from redisvl.query.filter import Geo, GeoRadius

# within 10 km of San Francisco office
geo_filter = Geo("office_location") == GeoRadius(-122.4194, 37.7749, 10, "km")
v.set_filter(geo_filter)

result_print(index.query(v))

vector_distance,user,credit_score,age,job,office_location
0,john,high,18,engineer,"-122.4194,37.7749"
0,derrick,low,14,doctor,"-122.4194,37.7749"
0.266666650772,nancy,high,94,doctor,"-122.4194,37.7749"


In [15]:
# within 100 km Radius of San Francisco office
geo_filter = Geo("office_location") == GeoRadius(-122.4194, 37.7749, 100, "km")
v.set_filter(geo_filter)

result_print(index.query(v))

vector_distance,user,credit_score,age,job,office_location
0,john,high,18,engineer,"-122.4194,37.7749"
0,derrick,low,14,doctor,"-122.4194,37.7749"
0.109129190445,tyler,high,100,engineer,"-122.0839,37.3861"
0.158809006214,tim,high,12,dermatologist,"-122.0839,37.3861"
0.217882037163,taimur,low,15,CEO,"-122.0839,37.3861"
0.266666650772,nancy,high,94,doctor,"-122.4194,37.7749"
0.653301358223,joe,medium,35,dentist,"-122.0839,37.3861"


In [16]:
# not within 10 km Radius of San Francisco office
geo_filter = Geo("office_location") != GeoRadius(-122.4194, 37.7749, 10, "km")
v.set_filter(geo_filter)

result_print(index.query(v))

vector_distance,user,credit_score,age,job,office_location
0.109129190445,tyler,high,100,engineer,"-122.0839,37.3861"
0.158809006214,tim,high,12,dermatologist,"-122.0839,37.3861"
0.217882037163,taimur,low,15,CEO,"-122.0839,37.3861"
0.653301358223,joe,medium,35,dentist,"-122.0839,37.3861"


## Combining Filters

In this example, we will combine a numeric filter with a tag filter. We will search for users that are between the ages of 20 and 30 and have a job of "engineer".

### Intersection ("and")

In [17]:
t = Tag("credit_score") == "high"
low = Num("age") >= 18
high = Num("age") <= 100

combined = t & low & high

v = VectorQuery([0.1, 0.1, 0.5],
                "user_embedding",
                return_fields=["user", "credit_score", "age", "job",  "office_location"],
                filter_expression=combined)


result_print(index.query(v))

vector_distance,user,credit_score,age,job,office_location
0,john,high,18,engineer,"-122.4194,37.7749"
0.109129190445,tyler,high,100,engineer,"-122.0839,37.3861"
0.266666650772,nancy,high,94,doctor,"-122.4194,37.7749"


### Union ("or")

The union of two queries is the set of all results that are returned by either of the two queries. The union of two queries is performed using the `|` operator.

In [18]:
low = Num("age") < 18
high = Num("age") > 93

combined = low | high
v.set_filter(combined)

result_print(index.query(v))

vector_distance,user,credit_score,age,job,office_location
0,derrick,low,14,doctor,"-122.4194,37.7749"
0.109129190445,tyler,high,100,engineer,"-122.0839,37.3861"
0.158809006214,tim,high,12,dermatologist,"-122.0839,37.3861"
0.217882037163,taimur,low,15,CEO,"-122.0839,37.3861"
0.266666650772,nancy,high,94,doctor,"-122.4194,37.7749"


## Other Redis Queries

Sometimes there may be a case where RedisVL does not cover the explicit functionality required by the query either because of new releases that haven't been implemented in the client, or because of a very specific use case. In these cases, it is possible to use the ``SearchIndex.search`` method to execute query with a redis-py ``Query`` object or through a raw redis string.

For example

### Redis-Py

In [19]:
# Manipulate the Redis-py Query object
redis_py_query = v.query

# choose to sort by age instead of vector distance
redis_py_query.sort_by("age", asc=False)

# run the query with the ``SearchIndex.search`` method
result = index.search(redis_py_query, v.params)
result_print(result)

vector_distance,age,user,credit_score,job,office_location
0.109129190445,100,tyler,high,engineer,"-122.0839,37.3861"
0.266666650772,94,nancy,high,doctor,"-122.4194,37.7749"
0.217882037163,15,taimur,low,CEO,"-122.0839,37.3861"
0,14,derrick,low,doctor,"-122.4194,37.7749"
0.158809006214,12,tim,high,dermatologist,"-122.0839,37.3861"


### Raw Redis Query String

So one case might be where you simply want to have a search that only filters on a tag field and don't need other functionality. Conversely, you may need to have a query that is more complex than what is currently supported by RedisVL. In these cases, you can use the ``SearchIndex.search`` again with just a raw redis query string.

In [20]:
t = Tag("credit_score") == "high"

str(t)

'@credit_score:{high}'

In [21]:
results = index.search(str(t))
for r in results.docs:
    print(r.__dict__)

{'id': 'v1:dc45946a8bc74f47858617c91d593b43', 'payload': None, 'user': 'john', 'age': '18', 'job': 'engineer', 'credit_score': 'high', 'office_location': '-122.4194,37.7749', 'user_embedding': '==\x00\x00\x00?'}
{'id': 'v1:5c628fdfbba247c6843955de04e3a00c', 'payload': None, 'user': 'nancy', 'age': '94', 'job': 'doctor', 'credit_score': 'high', 'office_location': '-122.4194,37.7749', 'user_embedding': '333?=\x00\x00\x00?'}
{'id': 'v1:4f1cb6dd167149d59c9c108e09407fc9', 'payload': None, 'user': 'tyler', 'age': '100', 'job': 'engineer', 'credit_score': 'high', 'office_location': '-122.0839,37.3861', 'user_embedding': '=>\x00\x00\x00?'}
{'id': 'v1:f1720dbeb81c4316bedf21ca60357fdf', 'payload': None, 'user': 'tim', 'age': '12', 'job': 'dermatologist', 'credit_score': 'high', 'office_location': '-122.0839,37.3861', 'user_embedding': '>>\x00\x00\x00?'}


## Inspecting Queries

In this example, we will show how to inspect the query that is generated by RedisVL. This can be useful for debugging purposes or for understanding how the query is being executed.

Let's again take the example of a query that combines a numeric filter with a tag filter. This will search for users that are between the ages of between 18 and 100, have a high credit score, and sort by closest vector distance to the query vector.

In [22]:
t = Tag("credit_score") == "high"
low = Num("age") >= 18
high = Num("age") <= 100

combined = t & low & high

v.set_filter(combined)

# Using the str() method, you can see what Redis Query this will emit.
str(v)

'((@credit_score:{high} @age:[18 +inf]) @age:[-inf 100])=>[KNN 10 @user_embedding $vector AS vector_distance] RETURN 6 user credit_score age job office_location vector_distance SORTBY vector_distance ASC DIALECT 2 LIMIT 0 10'